In [3]:
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.55.4
!pip install --no-deps trl==0.22.2

In [1]:
import sys
print(sys.executable)
print(sys.path)

C:\Users\jinwi\Programming-Files\email-llm\.venv\Scripts\python.exe
['C:\\Users\\jinwi\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'C:\\Users\\jinwi\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'C:\\Users\\jinwi\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'C:\\Users\\jinwi\\AppData\\Local\\Programs\\Python\\Python313', 'C:\\Users\\jinwi\\Programming-Files\\email-llm\\.venv', '', 'C:\\Users\\jinwi\\Programming-Files\\email-llm\\.venv\\Lib\\site-packages', 'C:\\Users\\jinwi\\Programming-Files\\email-llm\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\jinwi\\Programming-Files\\email-llm\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\jinwi\\Programming-Files\\email-llm\\.venv\\Lib\\site-packages\\Pythonwin']


In [1]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

CUDA available: True
Number of GPUs: 1
Current GPU: 0
GPU Name: NVIDIA GeForce RTX 4070 Laptop GPU


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096
dtype = None 
load_in_4bit = True 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B-Base-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

C:\Users\jinwi\Programming-Files\email-llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W1015 16:06:01.433000 103996 Lib\site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.3: Fast Qwen3 patching. Transformers: 4.55.4.
   \\   /|    NVIDIA GeForce RTX 4070 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Input:
    {}

    ### Response:
    {}"""
EOS_TK = tokenizer.eos_token
def format_inputs(batch):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_datase

PATH = '../' 
# "/content/drive/MyDrive/qwen-email-datasets/"

dataset = load_dataset("parquet", data_files={'train': f"{PATH}processed_emails.parquet", 'validation': f"{PATH}val_emails.parquet", 'test': f"{PATH}test_emails.parquet"})
dataset = dataset.map(format_inputs, batched=True)

In [ ]:
from trl import SFTConfig, SFTTrainer

args = SFTConfig(
    output_dir=f"./email-qwen3-14b-BASE",
    num_train_epochs=3,
    per_device_train_batch_size=3,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_8bit",
    save_strategy="steps",
    save_steps=250,
    eval_strategy="steps",
    eval_steps=250,
    learning_rate=5e-5,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    logging_steps=10,
    lr_scheduler_type="linear",
    push_to_hub=False,  
    load_best_model_at_end = True,
    report_to="none",
    metric_for_best_model = "eval_loss",
    greater_is_better = False,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    eval_dataset = dataset['validation'],
    max_seq_length = 10000,
    args = args,
)



In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "Instruction:\n",
    response_part = "Response:\n",
)